In [1]:
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 36301266044 (33.81 GB)
DFS Remaining: 35772280832 (33.32 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 1.46%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.20.0.5:9866 (cs544_tester_directory-dn-2.cs544_tester_directory_default)
Hostname: 04f80de43ac5
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 273559896 (260.89 MB)
Non DFS Used: 7644577448 (7.12 GB)
DFS Remaining: 17886138368 (16.66 GB)
DFS Used%: 1.06%
DFS

In [2]:
import time
time.sleep(30)

In [3]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv

cp: `hdfs://boss:9000/single.csv': File exists


In [4]:
import requests
import pyarrow as pa
import pyarrow.fs
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN"
container_number_count = dict()


for offset in range(0, 174944099, 1048576):
    url_with_offset = f"{url}&offset={offset}"
    r = requests.get(url_with_offset,stream = True)
    if r.status_code == 403:
        if "lost" not in container_number_count:
            container_number_count['lost'] = 1
        else:
            container_number_count['lost'] +=1
    if r.status_code == 200:
        url_array = r.url.split('/')
        # print(url_array)
        container_id = url_array[2]
        container = container_id.split(':')
        final_container_id = container[0]
        if final_container_id not in container_number_count:
            container_number_count[final_container_id] = 1
        else:
            container_number_count[final_container_id] += 1

print(dict(container_number_count))

{'04f80de43ac5': 92, 'lost': 75}


In [5]:
#q10


count = 0
good_blocks = []
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN"
for offset in range(0, 174944099, 1048576):
    url_with_offset = f"{url}&offset={offset}"
    r = requests.get(url_with_offset,stream = True)
    if r.status_code == 403:
        continue
    if r.status_code == 200:
        url_array = r.url
        good_blocks.append(url_array.split("=")[-1])


for i in good_blocks:
    hdfs = pa.fs.HadoopFileSystem("boss", 9000)
    with hdfs.open_input_file('hdfs://boss:9000/single.csv') as f:
        reader = f.read_at(1048576,int(i))
        count += str(reader).count("Single Family")
count

2023-10-27 01:09:24,401 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


245344